In [ ]:
DROP TABLE nppbuf.T_DD_FORECAST_DETAIL_FJHIS;
CREATE TABLE nppbuf.T_DD_FORECAST_DETAIL_FJHIS AS   
SELECT dh.hostpartid,                                 --零件号    
       dh.hostlocid,                                  --仓库代码  
       dh.historybegdate,                             --年月日  
       dh.dshostid,                                     --需求流  
       dh.historyamount,                              --需求数量  
       sa.abc_hits_network,                           --ABC需求频次分类  
       sa.abc_cost,                                   --ABC价格分类  
       sa.abc_demand,                                 --ABC需求数量分类  
       (substr(to_char(dh.hostpartid), 4, 1)) AS aa,          --零件号第四位  
       (CASE WHEN kb.hostkittingid IS NOT NULL THEN 'Y' ELSE 'N' END) AS bb ,    --是否组装包  
       (CASE WHEN SKU.HOSTPLANNERCODEID = '006'   
             AND (SKU.hostpartid LIKE 'ZCX%' OR SKU.hostpartid LIKE 'P%')    
             THEN 'N'   
             ELSE 'Y'   
       END) AS cc  --组装包是否参与预测  
  FROM (SELECT * FROM nppbuf.t_dd_demand_detail  
        UNION ALL   
        SELECT * FROM nppbuf.t_dd_demand_detail_history) dh  
  LEFT JOIN nppcore.ipcs_part_master pm  
    ON dh.hostpartid = pm.partnumber  
  LEFT JOIN nppcore.ipcs_loc_master lm  
    ON dh.hostlocid = lm.hostlocid  
  LEFT JOIN nppcore.ipcscust_stock_amount sa  
    ON pm.partid = sa.partid  
   AND lm.locid = sa.locid  
  LEFT JOIN nppbuf.t_dd_part_kit_bom kb  
    ON dh.hostpartid = substr(kb.hostkittingid, 6)  
   AND substr(dh.hostlocid, 2, 1) = substr(kb.hostkittingid, 2, 1)  
  LEFT JOIN nppbuf.T_DD_SKU SKU  
    ON dh.hostpartid = SKU.hostpartid  
   AND dh.hostlocid = SKU.hostlocid;
   
TRUNCATE TABLE nppbuf.T_DD_FORECAST_DETAIL_test 

CREATE TABLE nppbuf.t_dd_demand_outlier_trade AS  
SELECT DISTINCT HOSTPARTID ,HOSTLOCID ,DSHOSTID ,HISTORYBEGDATE ,HISTORYAMOUNT  FROM nppbuf.t_dd_demand_detail 

In [ ]:
TRUNCATE TABLE nppbuf.T_DD_FORECAST_DETAIL_test 

show table nppbuf.T_DD_FORECAST_DETAIL_test 

In [ ]:

CREATE TABLE nppbuf.T_DD_DEMAND_remove_mmx AS
WITH grouped_data AS (
  SELECT
    HOSTPARTID,
    HOSTLOCID,
    DSHOSTID,
    HISTORYAMOUNT,
    ROW_NUMBER() OVER (PARTITION BY HOSTPARTID, HOSTLOCID, DSHOSTID ORDER BY HISTORYAMOUNT DESC) AS row_num
  FROM nppbuf.t_dd_demand_detail -- 将表名替换为实际表名
)
SELECT
  HOSTPARTID,
  HOSTLOCID,
  DSHOSTID,
  HISTORYAMOUNT
FROM grouped_data
WHERE row_num > 1;



CREATE TABLE nppbuf.T_DD_DEMAND_DETAIL_outlier AS 
SELECT HOSTPARTID,HOSTLOCID,DSHOSTID, 
CASE   
   WHEN 3 * STDDEV_POP(HISTORYAMOUNT) < 20 THEN 20   
   ELSE 3 * STDDEV_POP(HISTORYAMOUNT)   
 END AS three_stddev  

FROM nppbuf.T_DD_DEMAND_remove_mmx 
GROUP BY HOSTPARTID,HOSTLOCID,DSHOSTID

In [ ]:
# 异常订单
CREATE TABLE nppbuf.T_DD_DEMAND_combined AS
WITH grouped_data AS (
  SELECT
    HOSTPARTID,
    HOSTLOCID,
    DSHOSTID,
    HISTORYAMOUNT,
    ROW_NUMBER() OVER (PARTITION BY HOSTPARTID, HOSTLOCID, DSHOSTID ORDER BY HISTORYAMOUNT DESC) AS row_num
  FROM nppbuf.t_dd_demand_detail -- 将表名替换为实际表名
)
, filtered_data AS (
  SELECT
    HOSTPARTID,
    HOSTLOCID,
    DSHOSTID,
    HISTORYAMOUNT
  FROM grouped_data
  WHERE row_num > 1
)
SELECT
  f.HOSTPARTID,
  f.HOSTLOCID,
  f.DSHOSTID,
  CASE
    WHEN 3 * STDDEV_POP(f.HISTORYAMOUNT) < 20 THEN 20
    ELSE 3 * STDDEV_POP(f.HISTORYAMOUNT)
  END AS three_stddev
FROM filtered_data f
GROUP BY f.HOSTPARTID, f.HOSTLOCID, f.DSHOSTID;

In [ ]:
#历史需求统计
CREATE TABLE nppbuf.T_DD_DEMAND_hist_fj AS
SELECT     
    HOSTPARTID,    
    HOSTLOCID,    
    DSHOSTID,    
    SUBSTR(HISTORYBEGDATE, 1, 4) AS year,  
    SUBSTR(HISTORYBEGDATE, 5, 2) AS month ,
    SUM(HISTORYAMOUNT) AS TOTAL_HISTORYAMOUNT    
FROM     
    nppbuf.T_DD_FORECAST_DETAIL_FJHIS       
GROUP BY     
    HOSTPARTID,    
    HOSTLOCID,    
    DSHOSTID, SUBSTR(HISTORYBEGDATE, 1, 4),
  SUBSTR(HISTORYBEGDATE, 5, 2)